In [1]:
import pandas as pd
import numpy as np
from datetime import date, datetime as dt


def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items. Cantidad de resenas totales independientemente si son True o false y los juegos totales.

In [2]:
def userdata( User_id : str ):
    df_usu_games = pd.read_csv(r'df_games_userid.csv')
    items_usu_games = pd.read_csv(r'df_items_user_sample.csv')
    reviews_usu_games = pd.read_csv(r'df_reviews_userid.csv')
    
    games_usu = items_usu_games[items_usu_games['user_id'] == User_id]['item_id'] # Se lista la cantidad de juegos por usuario
    #gaames_usu = games_usu.astype(int)
    games_usu = games_usu.tolist() # Se crea una lista para recorrer con los id de los juegos filtrados por usuario

    # funcion para sumar el gasto

    prices = []
    for elemento in games_usu:
        if df_usu_games[df_usu_games['id'] == (elemento)]['price'].to_list() == []:
            pass
        else:
            prices.append(df_usu_games[df_usu_games['id'] == (elemento)]['price'].to_list()[0])

    suma = sum(prices)


    juegos_totales = items_usu_games.loc[items_usu_games['user_id'] == User_id, 'items_count'].tolist()[0]
    recomendaciones_f = reviews_usu_games[reviews_usu_games['user_id'] == User_id]['recommend'].tolist()
    recomendations = sum(recomendaciones_f)

    recomend = round((recomendations/juegos_totales)*100,2)
    
    diccionario = { 'Gasto': suma,
                   'Recomendacion' : recomend}

    return diccionario

        
userdata('76561198251004808')

{'Gasto': 14.99, 'Recomendacion': 5.0}

def countreviews( YYYY-MM-DD y YYYY-MM-DD : str ): Cantidad de usuarios que realizaron reviews entre las fechas dadas y, el porcentaje de recomendación de los mismos en base a reviews.recommend.

In [3]:
def countreviews(fecha_inicial, fecha_final):
    reviews_countreviews = pd.read_csv(r'df_reviews_countreviews.csv')
    lista_fechas = []
    lista_usuarios_unico = []
    inicial = dt.strptime(fecha_inicial, '%Y-%m-%d').date()
    final = dt.strptime(fecha_final, '%Y-%m-%d').date()

    for ind, elemento in enumerate(reviews_countreviews['posted']):
        elemento_fecha = dt.strptime(elemento, '%Y-%m-%d').date() 
        if inicial <= elemento_fecha <= final:
            lista_fechas.append(elemento_fecha)
            a = reviews_countreviews['user_id'][ind]
            if a not in lista_usuarios_unico:
                lista_usuarios_unico.append(a)

    cuenta_usuarios = len(lista_usuarios_unico)

# Hago nuevamente el codigo pero sin considererar si el usuario ya estaba en la lista, esto permite saber cuantas veces hizo reviews en total
    lista_usuarios = []
    lista_fila = []
    numero_fila = 0
    for ind, elemento in enumerate(reviews_countreviews['posted']):
        elemento_fecha = dt.strptime(elemento, '%Y-%m-%d').date() 
        if inicial <= elemento_fecha <= final:
            a = reviews_countreviews['user_id'][ind] # Indica el user id dentro del rango de fechas
            lista_usuarios.append(a) # Lista para guardar los Id's y posteriormente saber cuantos usuarios hay (Dado que el numero de usuarios es igual al de recomendaciones se usa como denominaror)
            numero_fila = ind
            lista_fila.append(numero_fila) # Esta lista de filas se usa posteriormente para saber cuantas recomendaciones fueron positivas
    
    b = 0
    for elemento in lista_fila:
        if reviews_countreviews['recommend'][elemento] == True:
            b += 1 # Numero de recomendaciones positivas

    recomendacion = round(b/len(lista_fila)*100,2)
    
    return {'Usuarios_rango' : cuenta_usuarios, 
            'recomendacion %' : f"{recomendacion:.2f}%"
            }



countreviews('2015-01-01', '2015-12-31')

{'Usuarios_rango': 10951, 'recomendacion %': '85.52%'}

def genre( género : str ): Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

In [4]:
def genre( género : str ):

    merged = pd.read_csv(r'merged_genres.csv')
    # lista de generos
    generos = merged['genres'].unique()

    # defino funcion para calcular el total jugado por genero
    listacalc = []
    def tiempo_genero(lista):
        listacalc = []
        for elemento in lista:
            calculo = merged.loc[merged['genres'] == str(elemento), 'playtime_forever'].sum()
            listacalc.append(calculo)
        return listacalc

    a = tiempo_genero(generos)
    
    # creo un dataframe con las dos listas generadas

    datos = {
        'generos' : generos,
        'tiempo_juego': a
         }

    df = pd.DataFrame(datos)

    # Creo una columna con el ranking

    df['ranking'] = df['tiempo_juego'].rank(ascending=False)

    condicion = str(género)
    ranking = df.loc[df['generos'] == str(género), 'ranking'].values[0]


    diccionario = { 'Genero': condicion,
                   'Ranking': ranking
    }
    return diccionario

genre('Casual')
    


{'Genero': 'Casual', 'Ranking': 7.0}

def userforgenre( género : str ): Top 5 de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.

In [5]:
def userforgenre(género: str):
    games_gen2 = pd.read_csv(r'df_games_userforgente.csv')
    items = pd.read_csv(r"df_items_userforgenre.csv")
    #games_gen2['id'] = games_gen2['id'].astype(str)
    genero = items.merge(games_gen2, left_on='item_id', right_on='id')
    genero = genero[genero['genres'] == género]
    user_time = genero.groupby('user_id')['playtime_forever'].sum()
    usuarios_top = user_time.sort_values(ascending=False).head(5)
    ranking = []
    for user_id, horas in usuarios_top.items():
        usuario = items[items['user_id'] == user_id].iloc[0]
        ranking.append([usuario['user_id'], f"http://steamcommunity.com/id/{usuario['user_id']}",horas/60])
    return { 'Resultado': ranking
                }

userforgenre('Action')

{'Resultado': [['76561197977470391',
   'http://steamcommunity.com/id/76561197977470391',
   8478.416666666666],
  ['laumir', 'http://steamcommunity.com/id/laumir', 7132.8],
  ['ILDHAM123', 'http://steamcommunity.com/id/ILDHAM123', 6443.166666666667],
  ['LapFucksTrax', 'http://steamcommunity.com/id/LapFucksTrax', 5765.85],
  ['76561197984830126',
   'http://steamcommunity.com/id/76561197984830126',
   5601.983333333334]]}

def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de salida:
Activision	
Año	Contenido Free
2023	27%
2022	25%
xxxx	xx%

In [6]:
def developer( desarrollador : str ):
    games_dev = pd.read_csv(r'df_games_developer.csv')
    pd.set_option('display.float_format', '{:.2f}'.format) # Seteo para permitir mostrar numeros muy grandes

# columna free not free para luego sacar el % con base en la cantidad de juegos (items) por desarrollador, no se hace con item count pues este se refiere a los items por usuario y no por desarrollador
    games_dev['free_notfree'] = np.where(games_dev['price'] == 0,  True,  False)
# Filtro para aplicar la funcion cuando se de el developer
    developerdf = games_dev.loc[games_dev['developer'] == str(desarrollador)]
# conteo de cantidades de juegos(items) y free items por desarrollados

    cantidad_juegos_anio = developerdf.groupby('release_date_anio')['id'].count().reset_index()
    cantidad_juegos_anio.rename(columns={'id': 'cantidad_items'}, inplace=True)
# Conteo de la cantidad de juegos gratuitos (items) por año
    cantidad_free_anio = developerdf[developerdf['free_notfree']].groupby('release_date_anio')['free_notfree'].count().reset_index()
    cantidad_free_anio.rename(columns={'free_notfree': 'cantidad_free'}, inplace=True)
# Combino los resultados
    resultado_final = pd.merge(cantidad_juegos_anio, cantidad_free_anio, on='release_date_anio', how='outer')
# Rellenar valores NaN con 0 en la columna 'cantidad_free'
    resultado_final['cantidad_free'].fillna(0, inplace=True)
# Calcular el porcentaje de juegos gratuitos (items) por año
    resultado_final['porcentaje_free'] = (resultado_final['cantidad_free'] / resultado_final['cantidad_items']) * 100
# Formatear la columna 'porcentaje_free' con el símbolo de porcentaje (%)
    resultado_final['porcentaje_free'] = resultado_final['porcentaje_free'].apply(lambda x: f"{x:.2f}%")
    resultado_final['release_date_anio'] = resultado_final['release_date_anio'].apply(lambda x: f"{x:.0f}")
# Dejo solo las columnas para el return
    df_final = resultado_final[['release_date_anio','porcentaje_free']]    


    diccionario = {
        'Desarrollador': desarrollador,
        'Tabla': df_final[['release_date_anio','porcentaje_free']]
    }
    return diccionario

developer('Valve')

{'Desarrollador': 'Valve',
 'Tabla':    release_date_anio porcentaje_free
 0               1998           0.00%
 1               1999           0.00%
 2               2000           0.00%
 3               2001           0.00%
 4               2003           0.00%
 5               2004           0.00%
 6               2006           0.00%
 7               2007          50.00%
 8               2008           0.00%
 9               2009           0.00%
 10              2010          50.00%
 11              2011           0.00%
 12              2012           0.00%
 13              2017           0.00%}

def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

                    Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [7]:
def sentiment_analysis(year: int):
    merged = pd.read_csv(r'merged_recomendacion.csv')
    merged['release_date_anio'] = merged['release_date_anio'].astype(int)


    year = int(year)
    df_sa = merged.copy()
    df_sa['anio'] = df_sa['release_date_anio']
    anio_data = df_sa[df_sa['anio'] == year]
    mapeo = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
    sentiment = anio_data['sentiment'].map(mapeo).value_counts().to_dict()
    return sentiment

sentiment_analysis(2010)

{'Positive': 1954, 'Negative': 452, 'Neutral': 4}